In [1]:

# Core Libraries for Data Manipulation and Numerical Computation
import pandas as pd
import numpy as np

# Visualisation Libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Render plotly graph on GitHub
import plotly.io as pio
pio.renderers.default = 'notebook+png'

# Balance  size and resolution of plotly graphs
pio.defaults.width = 1100
pio.defaults.height = 450
pio.defaults.scale = 1.5  


# Time Series Utlities
import calendar

# Database Interraction
import sqlite3

# Expression Evaluation
import ast

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the dataset
tn_df = pd.read_csv("unzippedData\\tn.movie_budgets.csv", encoding = 'Latin1')
tn_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [3]:
tn_df.shape

(5782, 6)

In [4]:
# Load gross dataset
bom_df = pd.read_csv("unzippedData\\bom.movie_gross.csv")
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [5]:
bom_df.shape

(3387, 5)

In [6]:
null_counts = bom_df.isna().sum()
null_percentages = (null_counts / len(bom_df)) * 100

print("Missing Values Overview:\n")
print(pd.concat([null_counts.rename("Null Count"), null_percentages.round(2).rename("Null Percentage (%)")], axis = 1))

Missing Values Overview:

                Null Count  Null Percentage (%)
title                    0                 0.00
studio                   5                 0.15
domestic_gross          28                 0.83
foreign_gross         1350                39.86
year                     0                 0.00


In [7]:
# Get rows with nulls
bom_df.loc[bom_df['studio'].isna()]

,title,studio,domestic_gross,foreign_gross,year
210,Outside the Law (Hors-la-loi),NaN,96900.0,3300000,2010
555,Fireflies in the Garden,NaN,70600.0,3300000,2011
933,Keith Lemon: The Film,NaN,NaN,4000000,2012
1862,Plot for Peace,NaN,7100.0,NaN,2014
2825,Secret Superstar,NaN,NaN,122000000,2017


In [8]:
# Impute nulls in studio column
bom_df.loc[bom_df['title'] == 'Plot for Peace', 'studio'] = 'Trinity Films'
bom_df.loc[bom_df['title'] == 'Fireflies in the Garden', 'studio'] = 'Senator Entertainmet Inc.'
bom_df.loc[bom_df['title'] == 'Keith Lemon: The Film', 'studio'] = 'Lionsgate'
bom_df.loc[bom_df['title'] == 'Secret Superstar', 'studio'] = 'Zee Studios'
bom_df.loc[bom_df['title'] == 'Outside the Law (Hors-la-loi)', 'studio'] = 'StudioCanal'

# Preview changes
print("Null values after imputation:", bom_df['studio'].isna().sum())

Null values after imputation: 0


In [9]:
# Get rows with nulls
bom_df.loc[bom_df['domestic_gross'].isna()].head()

,title,studio,domestic_gross,foreign_gross,year
230,It's a Wonderful Afterlife,UTV,NaN,1300000,2010
298,Celine: Through the Eyes of the World,Sony,NaN,119000,2010
302,White Lion,Scre.,NaN,99600,2010
306,Badmaash Company,Yash,NaN,64400,2010
327,Aashayein (Wishes),Relbig.,NaN,3800,2010


In [10]:
median_gross = bom_df['domestic_gross'].median()
bom_df['domestic_gross'] = bom_df['domestic_gross'].fillna(median_gross)

print("Null Values after imputation:", bom_df['domestic_gross'].isna().sum())

Null Values after imputation: 0


In [11]:
# Preview rows with nulls in foreign_gross
bom_df.loc[bom_df['foreign_gross'].isna()].head()

,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010


In [12]:
# Inspect unique values in foreign_gross
bom_df.loc[~bom_df['foreign_gross'].astype(str).str.replace(',', '').str.isnumeric(), 'foreign_gross'].unique()

array([nan, '1,131.6', '1,019.4', '1,163.0', '1,010.0', '1,369.5'],
      dtype=object)

In [13]:
# Imputation strategy:

# Coerce to numeric
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')

# Impute median
median_foreign_gross = bom_df['foreign_gross'].median()
bom_df['foreign_gross'] = bom_df['foreign_gross'].fillna(median_foreign_gross)

# Preview changes
print("Null Values after imputation:", bom_df['foreign_gross'].isna().sum())

Null Values after imputation: 0


In [14]:
#!pip install rapidfuzz

In [15]:
# import pandas as pd
# import re
# from rapidfuzz import process, fuzz

# # -------------------------------
# # 2) Helper: Clean titles
# # -------------------------------
# def clean_title(title):
#     if pd.isna(title):
#         return ""
#     title = title.lower()
#     title = re.sub(r'\([^)]*\)', '', title)  # remove (year) etc.
#     title = re.sub(r'[^a-z0-9\s]', '', title)  # remove punctuation
#     title = re.sub(r'\s+', ' ', title)  # collapse spaces
#     return title.strip()

# # -------------------------------
# # 3) Prepare data
# # -------------------------------
# bom_df["clean_title"] = bom_df["title"].apply(clean_title)

# tn_df["clean_title"] = tn_df["movie"].apply(clean_title)
# tn_df["release_date"] = pd.to_datetime(tn_df["release_date"], errors="coerce")
# tn_df["year"] = tn_df["release_date"].dt.year

# # BOM year is in the "year" column already
# bom_df["year"] = pd.to_numeric(bom_df["year"], errors="coerce")

# # Merge keys
# bom_df["merge_key"] = bom_df["clean_title"] + "_" + bom_df["year"].astype("Int64").astype(str)
# tn_df["merge_key"] = tn_df["clean_title"] + "_" + tn_df["year"].astype("Int64").astype(str)

# # -------------------------------
# # 4) Exact merge first
# # -------------------------------
# merged_exact = pd.merge(
#     bom_df,
#     tn_df,
#     on="merge_key",
#     how="inner",
#     suffixes=("_bom", "_tn")
# )

# # -------------------------------
# # 5) Fuzzy match the leftovers
# # -------------------------------
# bom_unmatched = bom_df[~bom_df["merge_key"].isin(merged_exact["merge_key"])]
# tn_keys = tn_df["merge_key"].dropna().tolist()

# matches = []
# for key in bom_unmatched["merge_key"]:
#     match, score, _ = process.extractOne(key, tn_keys, scorer=fuzz.token_sort_ratio)
#     if score >= 85:  # threshold tweakable
#         matches.append((key, match, score))

# fuzzy_map = pd.DataFrame(matches, columns=["merge_key_bom", "merge_key_tn", "score"])

# # -------------------------------
# # 6) Merge fuzzy matches back
# # -------------------------------
# fuzzy_merged = (
#     fuzzy_map
#     .merge(
#         bom_df,
#         left_on="merge_key_bom",
#         right_on="merge_key",
#         how="left"
#     )
#     .drop(columns=["merge_key_bom", "merge_key"])  # avoid duplicate key names
#     .merge(
#         tn_df,
#         left_on="merge_key_tn",
#         right_on="merge_key",
#         how="left",
#         suffixes=("_bom", "_tn")
#     )
#     .drop(columns=["merge_key_tn", "merge_key"])  # clean up
# )

# # -------------------------------
# # 7) Combine results
# # -------------------------------
# final_merged = pd.concat([merged_exact, fuzzy_merged], ignore_index=True)

# print(f"Exact matches: {len(merged_exact)}")
# print(f"Fuzzy matches: {len(fuzzy_merged)}")
# print(f"Total merged: {len(final_merged)}")

# # -------------------------------
# # 8) Save combined dataset
# # -------------------------------
# final_merged.to_csv("movies_merged.csv", index=False)
# print("Merged dataset saved to movies_merged.csv")


In [16]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Utility functions

def clean_title(title: str) -> str:

    """
        Lowercase, strip punctuation, parentheses, and extra spaces.
    """

    if pd.isna(title):
        return ""
    
    title = title.lower()
    title = re.sub(r'\([^)]*\)', '', title)       # remove parentheses + contents
    title = re.sub(r'[^a-z0-9\s]', '', title)     # remove punctuation
    title = re.sub(r'\s+', ' ', title)            # collapse spaces

    return title.strip()

def prepare_dataframe(df: pd.DataFrame, title_col: str, date_col: str = None) -> pd.DataFrame:

    """
        Clean titles, extract year if date column is given, and create merge key.
    """

    df = df.copy()
    df["clean_title"] = df[title_col].apply(clean_title)

    if date_col:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df["year"] = df[date_col].dt.year
    elif "year" not in df.columns:
        raise ValueError("No year column found — either provide a date_col or a 'year' column.")
    df["merge_key"] = df["clean_title"] + "_" + df["year"].astype(str)

    return df

def fuzzy_match_keys(source_keys, target_keys, threshold=85):

    """
        Return list of fuzzy matches above threshold.
    """

    matches = []

    for key in source_keys:
        match, score, _ = process.extractOne(key, target_keys, scorer=fuzz.token_sort_ratio)
        if score >= threshold:
            matches.append((key, match, score))

    return matches

# Prep datasets

bom_df_clean = prepare_dataframe(bom_df, title_col="title")
tn_df_clean  = prepare_dataframe(tn_df, title_col="movie", date_col="release_date")

# Exact merge
merged_exact = pd.merge(
    bom_df_clean, tn_df_clean,
    on="merge_key", how="inner", suffixes=('_bom', '_tn')
)

# Fuzzy match for non-matches
bom_unmatched = bom_df_clean.loc[~bom_df_clean["merge_key"].isin(merged_exact["merge_key"])]
tn_keys = tn_df_clean["merge_key"].tolist()

fuzzy_pairs = fuzzy_match_keys(bom_unmatched["merge_key"], tn_keys, threshold=85)
fuzzy_map = pd.DataFrame(fuzzy_pairs, columns=["merge_key_bom", "merge_key_tn", "score"])

# Merge fuzzy matches with BOM dataset first
fuzzy_merged = (
    fuzzy_map
    .merge(bom_df_clean, left_on="merge_key_bom", right_on="merge_key", suffixes=("", "_bom"))
    .drop(columns=["merge_key"])  # drop to avoid MergeError
    .merge(tn_df_clean, left_on="merge_key_tn", right_on="merge_key", suffixes=("_bom", "_tn"))
)

# Combine & report
final_merged = pd.concat([merged_exact, fuzzy_merged], ignore_index=True)

print(f"Exact matches: {len(merged_exact)}")
print(f"Fuzzy matches: {len(fuzzy_merged)}")
print(f"Total merged: {len(final_merged)}")
print(f"Coverage BOM: {round(len(final_merged) / len(bom_df) * 100, 2)}%")
print(f"Coverage TN : {round(len(final_merged) / len(tn_df) * 100, 2)}%")

# final_merged now holds all matches

Exact matches: 1418
Fuzzy matches: 129
Total merged: 1547
Coverage BOM: 45.67%
Coverage TN : 26.76%


In [17]:
final_merged

,title,studio,domestic_gross_bom,foreign_gross,year_bom,clean_title_bom,merge_key,id,release_date,movie,production_budget,domestic_gross_tn,worldwide_gross,clean_title_tn,year_tn,merge_key_bom,merge_key_tn,score
0,Toy Story 3,BV,415000000.0,652000000.0,2010,toy story 3,toy story 3_2010,47,2010-06-18,Toy Story 3,"$200,000,000","$415,004,880","$1,068,879,522",toy story 3,2010,NaN,NaN,NaN
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,alice in wonderland,alice in wonderland_2010,51,2010-03-05,Alice in Wonderland,"$200,000,000","$334,191,110","$1,025,491,110",alice in wonderland,2010,NaN,NaN,NaN
2,Inception,WB,292600000.0,535700000.0,2010,inception,inception_2010,38,2010-07-16,Inception,"$160,000,000","$292,576,195","$835,524,642",inception,2010,NaN,NaN,NaN
3,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,shrek forever after,shrek forever after_2010,27,2010-05-21,Shrek Forever After,"$165,000,000","$238,736,787","$756,244,673",shrek forever after,2010,NaN,NaN,NaN
4,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000.0,2010,the twilight saga eclipse,the twilight saga eclipse_2010,53,2010-06-30,The Twilight Saga: Eclipse,"$68,000,000","$300,531,751","$706,102,828",the twilight saga eclipse,2010,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,The Big Lebowski (20th Anniversary),Fathom,583000.0,18900000.0,2018,the big lebowski,the big lebowski_1998,75,1998-03-06,The Big Lebowski,"$15,000,000","$17,498,804","$46,189,568",the big lebowski,1998,the big lebowski_2018,the big lebowski_1998,90.476190
1543,The Guardians,MBox,177000.0,18900000.0,2018,the guardians,the guardian_2006,22,2006-09-29,The Guardian,"$70,000,000","$55,011,732","$94,973,540",the guardian,2006,the guardians_2018,the guardian_2006,85.714286
1544,Unstoppable (2018),WGUSA,101000.0,18900000.0,2018,unstoppable,unstoppable_2010,19,2010-11-12,Unstoppable,"$95,000,000","$81,562,942","$165,720,921",unstoppable,2010,unstoppable_2018,unstoppable_2010,93.750000
1545,The Apparition (2018),MBox,28300.0,18900000.0,2018,the apparition,the apparition_2012,4,2012-08-24,The Apparition,"$17,000,000","$4,936,819","$10,637,281",the apparition,2012,the apparition_2018,the apparition_2012,94.736842


In [18]:
final_merged.columns

Index(['title', 'studio', 'domestic_gross_bom', 'foreign_gross', 'year_bom',
       'clean_title_bom', 'merge_key', 'id', 'release_date', 'movie',
       'production_budget', 'domestic_gross_tn', 'worldwide_gross',
       'clean_title_tn', 'year_tn', 'merge_key_bom', 'merge_key_tn', 'score'],
      dtype='object')

In [ ]:
# Drop helper columns
final_clean = final_merged.drop(columns=[
    "clean_title_bom", "clean_title_tn",
    "merge_key", "merge_key_bom", "merge_key_tn", "score"
], errors="ignore")

# Rename columns to be consistent & descriptive
final_clean = final_clean.rename(columns={
    "title": "title_bom",
    "movie": "title_tn",
    "studio": "studio_bom",
    "domestic_gross_bom": "domestic_gross_bom_usd",
    "foreign_gross": "foreign_gross_bom_usd",
    "year_bom": "year_bom",
    "production_budget": "production_budget_usd",
    "domestic_gross_tn": "domestic_gross_tn_usd",
    "worldwide_gross": "worldwide_gross_tn_usd",
    "year_tn": "year_tn"
})

# Save cleaned version
final_clean.to_csv("movies_merged_clean.csv", index=False)
print("Cleaned merged dataset saved to movies_merged_clean.csv")
